In [1]:
import pandas as pd
import numpy as np
import ftfy
import math

In [2]:
name_dict_trusted = np.load('name_dict.npy',allow_pickle='TRUE').item()
code_dict_trusted = np.load('code_dict.npy',allow_pickle='TRUE').item()
code_dict_risky = np.load('code_dict_risky.npy',allow_pickle='TRUE').item()

In [3]:
try: 
    name_dict_trusted.pop('')
except: 
    pass
try: 
    code_dict_trusted.pop('')
except: 
    pass
try: 
    code_dict_risky.pop('')
except: 
    pass

### Complement missing airport name & code using latitude & longitude

In [4]:
data = pd.read_csv("NTSB_AIDS_full.txt", sep="|")

ntsb_code = pd.read_csv("NTSB_airportcode.csv")
ntsb_name = pd.read_csv("NTSB_airportname.csv")
airport_code = pd.read_csv("airports.csv", usecols=['ident', 'type', 'name', 'latitude_deg', 'longitude_deg',
       'iso_country', 'iso_region', 'municipality', 'iata_code', 'local_code'])

In [5]:
airport_code['latitude_deg_rounded'] = np.round(airport_code['latitude_deg'], decimals=1)
airport_code['longitude_deg_rounded'] = np.round(airport_code['longitude_deg'], decimals=1)

In [6]:
airport_code

,ident,type,name,latitude_deg,longitude_deg,iso_country,iso_region,municipality,iata_code,local_code,latitude_deg_rounded,longitude_deg_rounded
0,00A,heliport,Total Rf Heliport,40.070801,-74.933601,US,US-PA,Bensalem,NaN,00A,40.1,-74.9
1,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,US,US-KS,Leoti,NaN,00AA,38.7,-101.5
2,00AK,small_airport,Lowell Field,59.949200,-151.695999,US,US-AK,Anchor Point,NaN,00AK,59.9,-151.7
3,00AL,small_airport,Epps Airpark,34.864799,-86.770302,US,US-AL,Harvest,NaN,00AL,34.9,-86.8
4,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,US,US-AR,Newport,NaN,NaN,35.6,-91.3
...,...,...,...,...,...,...,...,...,...,...,...,...
56172,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,CN,CN-21,Yingkou,YKH,NaN,40.5,122.4
56173,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,CN,CN-21,Shenyang,NaN,NaN,41.8,123.5
56174,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,GB,GB-ENG,Sealand,NaN,NaN,51.9,1.5
56175,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,TF,TF-U-A,Grande Glorieuse,NaN,NaN,-11.6,47.3


In [7]:
new_lat = []
new_lon = []
for lat in data[' Latitude ']:
    try:
        new_lat.append(np.round(float(lat.strip()), decimals=1))
    except: 
        new_lat.append(999999)
for lon in data[' Longitude ']: 
    try:
        new_lon.append(np.round(float(lon.strip()), decimals=1))
    except: 
        new_lon.append(999999)
data['latitude_rounded'] = new_lat
data['longitude_rounded'] = new_lon

In [8]:
data_merged = data.merge(airport_code, how='inner', left_on='latitude_rounded', right_on='latitude_deg_rounded')
data_merged = data_merged.loc[(data_merged['longitude_rounded'] == data_merged['longitude_deg_rounded']),:]

len(data_merged), len(data)

(41757, 83807)

In [9]:
# total = len(data_merged)
# count = 0
# for ind in data_merged.index: 
#     data.loc[ind, ' Airport Code '] = data_merged.loc[ind, 'ident']
#     data.loc[ind, ' Airport Name '] = data_merged.loc[ind, 'name']
#     count += 1
#     if count == round(0.1*total):
#         print("10% done")
#     if count == round(0.2*total):
#         print("20% done")
#     if count == round(0.3*total):
#         print("30% done")
#     if count == round(0.4*total):
#         print("40% done")
#     if count == round(0.5*total):
#         print("50% done")
#     if count == round(0.6*total):
#         print("60% done")
#     if count == round(0.7*total):
#         print("70% done")
#     if count == round(0.8*total):
#         print("80% done")
#     if count == round(0.9*total):
#         print("90% done")

### Merge clean airportcode and airportname fields back

#### Use data_merged

In [10]:
def isinvalid(entry): 
    if pd.isna(entry):
        return True
    if type(entry) == str: 
        if entry.strip() == "": 
            return True
        if entry.strip() == "N/A":
            return True
    return False

In [11]:
for ind in data.index:
    if isinvalid(data.loc[ind, ' Airport Name ']) or isinvalid(data.loc[ind, ' Airport Code ']):
        try: 
            search = data_merged.loc[(data_merged['latitude_deg_rounded'] == data.loc[ind, 'latitude_rounded'] and data_merged['longitude_deg_rounded'] == data.loc[ind, 'longitude_rounded']), 'ident']
            data.loc[ind, ' Airport Code '] = search.iloc[0, :]['ident']
            print(search)
        except: 
            pass

#### Clean format: code & name dataframe

In [12]:
def clean_paren(col):
    toReturn = []
    for string in col: 
        string = string.upper().strip()
        left = string.find('(')
        right = string.find(')')
        if left != -1 and right != -1: 
            try: 
                new_string = string[:left] + string[right+1:]
            except:
                new_string = string[:left]
            toReturn.append(new_string)
        elif left != -1: 
            new_string = string[:left]
            toReturn.append(new_string)
        else: 
            toReturn.append(string)
    return toReturn

def clean(col): 
    toReturn = []
    for string in col:
        toReturn.append(''.join([c.upper() for c in string if c.isalpha() or c.isnumeric()]))
    return toReturn

In [13]:
# Fix unicode

code_org = [str(d) for d in ntsb_code["airportcode"]]
code_fix_unicode = []
code_failed = []
for code in code_org:
    try:
        code_fix_unicode.append(ftfy.fix_text(code))
    except:
        code_failed.append(str(code))  

name_org = [str(d) for d in ntsb_name["airportname"]]
name_fix_unicode = []
name_failed = []
for code in name_org:
    try:
        name_fix_unicode.append(ftfy.fix_text(code))
    except:
        name_failed.append(str(code))  

ntsb_code["airportcode"] = clean(code_fix_unicode)
ntsb_name["airportname"] = [string.strip() for string in name_fix_unicode]

In [14]:
country_code = pd.read_csv('iso_3166_country_codes.csv', encoding='ISO-8859-1')
airport_code = airport_code.merge(country_code, how='left', left_on='iso_country', right_on='alpha2')
airport_code

,ident,type,name_x,latitude_deg,longitude_deg,iso_country,iso_region,municipality,iata_code,local_code,latitude_deg_rounded,longitude_deg_rounded,name_y,alpha2
0,00A,heliport,Total Rf Heliport,40.070801,-74.933601,US,US-PA,Bensalem,NaN,00A,40.1,-74.9,United States,US
1,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,US,US-KS,Leoti,NaN,00AA,38.7,-101.5,United States,US
2,00AK,small_airport,Lowell Field,59.949200,-151.695999,US,US-AK,Anchor Point,NaN,00AK,59.9,-151.7,United States,US
3,00AL,small_airport,Epps Airpark,34.864799,-86.770302,US,US-AL,Harvest,NaN,00AL,34.9,-86.8,United States,US
4,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,US,US-AR,Newport,NaN,NaN,35.6,-91.3,United States,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56172,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,CN,CN-21,Yingkou,YKH,NaN,40.5,122.4,China,CN
56173,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,CN,CN-21,Shenyang,NaN,NaN,41.8,123.5,China,CN
56174,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,GB,GB-ENG,Sealand,NaN,NaN,51.9,1.5,United Kingdom,GB
56175,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,TF,TF-U-A,Grande Glorieuse,NaN,NaN,-11.6,47.3,French Southern Territories,TF


#### Clean format: data

In [15]:
data['airportcode'] = clean(data[' Airport Code '])
data['airportname'] = [string.strip() for string in data[' Airport Name ']]
data["airportname_cleanedletter"] = clean(clean_paren(data['airportname']))

data.drop([' Airport Code ', ' Airport Name '], axis=1, inplace=True)

In [16]:
data.head()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Injury Severity,Aircraft Damage,...,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,,latitude_rounded,longitude_rounded,airportcode,airportname,airportname_cleanedletter
0,20190917X60952,Accident,GAA19CA552,09/17/2019,"Palmer, AK",United States,61.355278,-148.409444,Non-Fatal,Substantial,...,,,Preliminary,09/18/2019,,61.4,-148.4,,N/A,NA
1,20190916X74851,Accident,GAA19CA550,09/16/2019,"Reno, NV",United States,39.668056,-119.876389,Unavailable,Substantial,...,VMC,,Preliminary,09/18/2019,,39.7,-119.9,RTS,Reno/Stead,RENOSTEAD
2,20190917X92718,Incident,ENG19IA034,09/16/2019,"Côte-d'Or, France",France,,,Incident,,...,,,Preliminary,09/18/2019,,999999.0,999999.0,,,
3,20190916X82539,Accident,CEN19LA321,09/15/2019,,United States,28.866945,-89.302222,Non-Fatal,Destroyed,...,,MANEUVERING,Preliminary,09/18/2019,,28.9,-89.3,,,
4,20190912X05539,Accident,GAA19CA544,09/12/2019,"Hemet, CA",United States,33.734166,-117.022223,Non-Fatal,Substantial,...,,,Preliminary,09/18/2019,,33.7,-117.0,HMT,Hemet-Ryan,HEMETRYAN


In [17]:
name_dict_trusted

{'NORTHLITTLEROCKMUNI': [29024],
 'MONTREALSTHUBERT': [16332],
 'PERRYHOUSTONCOUNTYAIRPORT': [29302],
 'DELTEX': [48947],
 'TRICOUNTYAIRPORT': [3018, 25923, 26749],
 'LARAMIEREGIONAL': [28419],
 'NULATO': [39180],
 'GUAMINTERNATIONALAIRPORT': [24729, 32991],
 'BOEINGHELIPORT': [12079],
 'HARTLEEFIELD': [4725],
 'SAOPAULOPATRIARCAHELIPORT': [5005],
 'FORTMADISON': [27855],
 'FRONTRANGEAIRPORTCOLORADO': [24729, 32991],
 'HALLOCKMUNICIPALAIRPORT': [28072],
 'QUILMES': [41700],
 'AKRONFULTON': [26696],
 'LACHOLLA': [6075],
 'NEWRICHMOND': [30424],
 'NORTONAIRSTRIP': [22685],
 'ARNOLDPALMERREGIONALAIRPORT': [28424],
 'DULLESINTERNATIONALAIRPORT': [24729, 32991],
 'HELSINKIVANTAA': [18622],
 'SCHOLESINTERNATIONAL': [27972],
 'GRANITE': [53242],
 'DELRIOINTERNATIONAL': [27230],
 'CHARLOTTEDOUGLAS': [27044],
 'CORTEZMUNICIPALAIRPORT': [26994],
 'MAYAGUEZ': [45329],
 'SHIFLETFIELD': [9895],
 'WINN': [28539],
 'BALDWINMUNICIPAL': [26464],
 'DOWNTOWNHOLMANFIELD': [30655],
 'AIRCOWBOYAIRPORT': [24

#### Use other geolocation information (Round 1: latitude & longitude within 1 unit)

In [18]:
missing = []
for i in zip([isinvalid(entry) for entry in data['airportname']], [isinvalid(entry) for entry in data['airportcode']]):
    missing.append(i[0] and i[1])

In [19]:
data_missing = data.loc[missing, :]
data_missing.head()

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Injury Severity,Aircraft Damage,...,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,,latitude_rounded,longitude_rounded,airportcode,airportname,airportname_cleanedletter
0,20190917X60952,Accident,GAA19CA552,09/17/2019,"Palmer, AK",United States,61.355278,-148.409444,Non-Fatal,Substantial,...,,,Preliminary,09/18/2019,,61.4,-148.4,,N/A,NA
2,20190917X92718,Incident,ENG19IA034,09/16/2019,"Côte-d'Or, France",France,,,Incident,,...,,,Preliminary,09/18/2019,,999999.0,999999.0,,,
3,20190916X82539,Accident,CEN19LA321,09/15/2019,,United States,28.866945,-89.302222,Non-Fatal,Destroyed,...,,MANEUVERING,Preliminary,09/18/2019,,28.9,-89.3,,,
6,20190912X21831,Accident,CEN19LA316,09/12/2019,"Eustace, TX",United States,32.276667,-96.006667,Fatal(1),Substantial,...,VMC,CRUISE,Preliminary,09/18/2019,,32.3,-96.0,,N/A,NA
9,20190909X51212,Accident,CEN19LA309,09/09/2019,"WALKERTON, IN",United States,41.505000,-86.515556,Non-Fatal,Substantial,...,,MANEUVERING,Preliminary,09/11/2019,,41.5,-86.5,,,


In [20]:
data.loc[missing, :].head(30)

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Injury Severity,Aircraft Damage,...,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,,latitude_rounded,longitude_rounded,airportcode,airportname,airportname_cleanedletter
0,20190917X60952,Accident,GAA19CA552,09/17/2019,"Palmer, AK",United States,61.355278,-148.409444,Non-Fatal,Substantial,...,,,Preliminary,09/18/2019,,61.4,-148.4,,N/A,NA
2,20190917X92718,Incident,ENG19IA034,09/16/2019,"Côte-d'Or, France",France,,,Incident,,...,,,Preliminary,09/18/2019,,999999.0,999999.0,,,
3,20190916X82539,Accident,CEN19LA321,09/15/2019,,United States,28.866945,-89.302222,Non-Fatal,Destroyed,...,,MANEUVERING,Preliminary,09/18/2019,,28.9,-89.3,,,
6,20190912X21831,Accident,CEN19LA316,09/12/2019,"Eustace, TX",United States,32.276667,-96.006667,Fatal(1),Substantial,...,VMC,CRUISE,Preliminary,09/18/2019,,32.3,-96.0,,N/A,NA
9,20190909X51212,Accident,CEN19LA309,09/09/2019,"WALKERTON, IN",United States,41.505000,-86.515556,Non-Fatal,Substantial,...,,MANEUVERING,Preliminary,09/11/2019,,41.5,-86.5,,,
11,20190911X92611,Accident,WPR19FA256,09/08/2019,"La Grande, OR",United States,45.146389,-118.293333,Fatal(1),Destroyed,...,,CRUISE,Preliminary,09/17/2019,,45.1,-118.3,,N/A,NA
13,20190909X13121,Accident,CEN19FA307,09/08/2019,"Lakeview, SD",United States,43.039444,-100.805556,Fatal(2),Substantial,...,VMC,UNKNOWN,Preliminary,09/16/2019,,43.0,-100.8,,N/A,NA
15,20190909X03241,Accident,GAA19CA528,09/07/2019,"Lakeport, CA",United States,39.039444,-122.907778,Unavailable,Substantial,...,,,Preliminary,09/10/2019,,39.0,-122.9,,,
18,20190907X85739,Accident,CEN19FA305,09/07/2019,"Carpenter, WY",United States,41.030000,-104.246389,Fatal(1),Destroyed,...,VMC,CRUISE,Preliminary,09/10/2019,,41.0,-104.2,,N/A,NA
23,20190905X13813,Accident,WPR19FA250,09/04/2019,"Henefer, UT",United States,41.015278,-111.580834,Fatal(2),Substantial,...,VMC,MANEUVERING,Preliminary,09/17/2019,,41.0,-111.6,,N/A,NA


In [21]:
total = len(data)
count = 0

for ind in data.index: 
    count += 1
    if count == round(0.1*total):
        print("10% done")
    if count == round(0.2*total):
        print("20% done")
    if count == round(0.3*total):
        print("30% done")
    if count == round(0.4*total):
        print("40% done")
    if count == round(0.5*total):
        print("50% done")
    if count == round(0.6*total):
        print("60% done")
    if count == round(0.7*total):
        print("70% done")
    if count == round(0.8*total):
        print("80% done")
    if count == round(0.9*total):
        print("90% done")
    
        
    if not (isinvalid(data.loc[ind, 'airportname']) and isinvalid(data.loc[ind, 'airportcode'])): 
        continue
        
    lat = data.loc[ind, 'latitude_rounded']
    lon = data.loc[ind, 'longitude_rounded']
    
    searched = airport_code.loc[airport_code['latitude_deg'] <= math.ceil(lat), :] 
    searched = searched.loc[airport_code['latitude_deg'] >= math.floor(lat), :]
    searched = searched.loc[airport_code['longitude_deg'] <= math.ceil(lon), :] 
    searched = searched.loc[airport_code['longitude_deg'] >= math.floor(lon), :]
    
    if len(searched) == 1: 
        data.loc[ind, 'airportcode'] = searched.loc[searched.index[0], 'ident']
        data.loc[ind, 'airportname'] = searched.loc[searched.index[0], 'name_x']
    else: 
        for sind in searched.index: 
            try: 
                if searched.loc[sind, 'municipality'].lower() in data.loc[ind, ' Location '].lower(): 
    #                 print('match found')
                    data.loc[ind, 'airportcode'] = searched.loc[sind, 'ident']
                    data.loc[ind, 'airportname'] = searched.loc[sind, 'name_x']
                    break
            except: 
                pass


10% done
20% done
30% done
40% done
50% done
60% done
70% done
80% done
90% done


#### Use other geolocation information (Round 2: country & location match)

In [22]:
# airport_code['name_y'].lower() contains data[' Country '].lower()
# AND data[' Location '].lower() contains airport_code['municipality'].lower()

In [23]:
airport_code

,ident,type,name_x,latitude_deg,longitude_deg,iso_country,iso_region,municipality,iata_code,local_code,latitude_deg_rounded,longitude_deg_rounded,name_y,alpha2
0,00A,heliport,Total Rf Heliport,40.070801,-74.933601,US,US-PA,Bensalem,NaN,00A,40.1,-74.9,United States,US
1,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,US,US-KS,Leoti,NaN,00AA,38.7,-101.5,United States,US
2,00AK,small_airport,Lowell Field,59.949200,-151.695999,US,US-AK,Anchor Point,NaN,00AK,59.9,-151.7,United States,US
3,00AL,small_airport,Epps Airpark,34.864799,-86.770302,US,US-AL,Harvest,NaN,00AL,34.9,-86.8,United States,US
4,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,US,US-AR,Newport,NaN,NaN,35.6,-91.3,United States,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56172,ZYYK,medium_airport,Yingkou Lanqi Airport,40.542524,122.358600,CN,CN-21,Yingkou,YKH,NaN,40.5,122.4,China,CN
56173,ZYYY,medium_airport,Shenyang Dongta Airport,41.784401,123.496002,CN,CN-21,Shenyang,NaN,NaN,41.8,123.5,China,CN
56174,ZZ-0001,heliport,Sealand Helipad,51.894444,1.482500,GB,GB-ENG,Sealand,NaN,NaN,51.9,1.5,United Kingdom,GB
56175,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584278,47.296389,TF,TF-U-A,Grande Glorieuse,NaN,NaN,-11.6,47.3,French Southern Territories,TF


In [24]:
total = len(data)
count = 0

for ind in data.index: 
    count += 1
    if count == round(0.1*total):
        print("10% done")
    if count == round(0.2*total):
        print("20% done")
    if count == round(0.3*total):
        print("30% done")
    if count == round(0.4*total):
        print("40% done")
    if count == round(0.5*total):
        print("50% done")
    if count == round(0.6*total):
        print("60% done")
    if count == round(0.7*total):
        print("70% done")
    if count == round(0.8*total):
        print("80% done")
    if count == round(0.9*total):
        print("90% done")
    
        
    if not (isinvalid(data.loc[ind, 'airportname']) and isinvalid(data.loc[ind, 'airportcode'])): 
        continue

    lat = data.loc[ind, 'latitude_rounded']
    lon = data.loc[ind, 'longitude_rounded']
    country = data.loc[ind, ' Country '].strip()
    location = data.loc[ind, ' Location '].lower().strip()
    try: 
        municipality = location.split(',')[0].strip()
        state = location.split(',')[1].strip()
    except: 
        municipality = location
        state = location
    
    if lat == 999999 or lon == 999999: 
        # check country
        searched =  airport_code.loc[airport_code['name_y'] == country, :]
        
        # check municipality
        for sind in searched.index: 
            try: 
                if searched.loc[sind, 'municipality'].lower() in data.loc[ind, ' Location '].lower(): 
                    if searched.loc[sind, 'iso_region'].split('-')[1].lower() == state: 
                        print(ind)
                        data.loc[ind, 'airportcode'] = searched.loc[sind, 'ident']
                        data.loc[ind, 'airportname'] = searched.loc[sind, 'name_x']
                        break
            except: 
                pass


48
70
74
94
143
145
207
252
268
373
407
414
416
470
472
479
498
546
551
574
626
629
677
764
768
810
876
877
878
879
888
891
899
930
955
982
1054
1076
1121
1130
1466
1472
1486
1554
1588
1625
1640
1642
1677
1687
1767
1794
1824
1840
1942
1969
2084
2158
2172
2305
2321
2385
2425
2454
2482
2514
2844
2857
2866
3113
3170
3805
3887
3891
4238
4285
4377
4508
4537
4592
4689
4761
4783
5146
5472
5568
5730
6277
6296
7246
7353
7643
7724
10% done
8897
9618
9969
10068
10131
10264
10378
10516
10802
10894
10998
11274
11283
11321
11350
11359
11594
11717
11922
11990
12013
12053
12061
12231
12269
12296
12388
12394
12415
12416
12565
12734
12898
13376
13440
13571
13572
13777
13784
13938
14001
14075
14388
14486
14508
14509
14781
14782
14845
14846
14903
14917
14947
14977
15035
15036
15069
15070
15136
15137
15194
15195
15227
15342
15514
15646
15672
15709
15710
15729
15830
15831
15947
16119
16297
16325
16484
16650
16733
20% done
17084
17245
17331
17332
17398
17563
17587
17633
17645
17659
17865
17882
17893
18251
18

37282
37283
37286
37294
37295
37298
37301
37303
37304
37306
37313
37318
37321
37331
37337
37339
37343
37351
37357
37362
37366
37367
37370
37371
37375
37377
37381
37387
37392
37398
37400
37403
37410
37413
37416
37418
37420
37423
37426
37430
37435
37438
37439
37452
37456
37457
37464
37471
37473
37476
37477
37478
37481
37496
37497
37510
37512
37513
37515
37521
37525
37527
37533
37537
37538
37551
37560
37577
37578
37594
37597
37598
37601
37602
37607
37610
37616
37625
37626
37628
37630
37633
37639
37651
37652
37653
37665
37667
37671
37672
37677
37686
37688
37692
37698
37700
37703
37706
37707
37712
37716
37721
37725
37736
37743
37745
37755
37756
37758
37765
37768
37771
37773
37777
37778
37784
37786
37797
37807
37813
37819
37822
37823
37825
37830
37836
37847
37850
37851
37852
37861
37866
37874
37875
37881
37888
37890
37892
37896
37899
37901
37904
37906
37908
37909
37915
37916
37917
37918
37921
37922
37927
37928
37933
37935
37942
37943
37944
37949
37951
37952
37955
37957
37958
37960
37961
3797

42462
42463
42466
42467
42468
42470
42471
42481
42484
42485
42486
42490
42492
42494
42499
42500
42501
42505
42507
42511
42512
42515
42516
42517
42518
42519
42520
42531
42532
42537
42539
42544
42545
42551
42555
42556
42558
42560
42564
42566
42569
42571
42573
42574
42578
42581
42593
42598
42599
42600
42609
42614
42618
42622
42626
42627
42628
42630
42639
42640
42641
42642
42652
42653
42656
42657
42658
42664
42665
42670
42673
42675
42681
42687
42688
42692
42696
42697
42703
42707
42709
42711
42720
42726
42728
42729
42730
42736
42737
42739
42740
42743
42744
42749
42750
42752
42753
42754
42759
42765
42768
42769
42770
42771
42772
42773
42774
42777
42786
42787
42789
42792
42794
42799
42800
42810
42823
42826
42832
42836
42837
42839
42840
42846
42849
42860
42862
42874
42876
42879
42881
42886
42889
42895
42901
42902
42903
42905
42906
42910
42911
42912
42915
42920
42923
42930
42933
42934
42939
42940
42946
42947
42952
42954
42961
42962
42965
42967
42972
42976
42979
42981
42983
42991
42992
42994
4299

46814
46817
46818
46819
46823
46829
46830
46831
46832
46836
46840
46845
46859
46860
46861
46867
46876
46877
46878
46879
46882
46886
46888
46890
46894
46898
46899
46902
46911
46912
46918
46922
46925
46927
46928
46930
46934
46936
46937
46939
46940
46946
46947
46948
46952
46955
46956
46958
46960
46962
46965
46977
46978
46979
46981
46984
46985
46986
46991
46999
47003
47008
47009
47024
47030
47033
47040
47042
47043
47044
47045
47047
47049
47052
47056
47057
47058
47059
47060
47062
47065
47066
47067
47069
47077
47081
47082
47085
47087
47091
47092
47104
47107
47110
47112
47114
47118
47119
47129
47134
47140
47141
47145
47146
47147
47149
47157
47163
47166
47168
47171
47178
47182
47183
47185
47186
47188
47190
47192
47193
47197
47200
47201
47205
47206
47208
47210
47212
47217
47219
47221
47222
47225
47227
47228
47232
47234
47236
47237
47246
47247
47248
47251
47255
47256
47259
47260
47261
47266
47276
47280
47281
47282
47288
47294
47299
47300
47301
47303
47305
47311
47312
47313
47315
47319
47321
4732

51187
51191
51197
51201
51203
51204
51205
51207
51212
51215
51220
51221
51224
51225
51229
51231
51232
51233
51234
51236
51240
51242
51243
51250
51251
51253
51259
51260
51264
51275
51281
51283
51286
51287
51288
51293
51298
51303
51308
51309
51313
51326
51327
51328
51335
51338
51344
51355
51358
51360
51363
51365
51366
51369
51372
51374
51377
51378
51380
51384
51390
51391
51399
51401
51413
51425
51430
51435
51440
51442
51445
51446
51448
51451
51452
51459
51460
51468
51475
51476
51479
51482
51489
51490
51491
51494
51495
51496
51504
51505
51512
51517
51518
51520
51524
51531
51533
51534
51535
51541
51544
51545
51553
51558
51560
51565
51566
51580
51582
51586
51594
51595
51596
51601
51602
51603
51614
51615
51618
51619
51623
51626
51633
51637
51640
51641
51642
51652
51658
51663
51665
51671
51674
51675
51678
51681
51683
51699
51701
51703
51705
51707
51708
51709
51710
51719
51726
51728
51733
51736
51749
51754
51758
51760
51761
51762
51763
51770
51772
51777
51778
51783
51784
51789
51794
51797
5179

...

60692
60694
60695
60697
60704
60713
60715
60716
60717
60718
60721
60727
60733
60735
60740
60743
60744
60746
60748
60754
60768
60777
60778
60781
60788
60796
60797
60798
60802
60808
60809
60812
60829
60830
60835
60846
60847
60856
60857
60861
60865
60866
60868
60869
60871
60872
60876
60879
60882
60883
60887
60888
60895
60896
60901
60909
60910
60911
60916
60924
60926
60930
60939
60945
60953
60957
60968
60969
60971
60974
60980
60989
60990
60993
60994
60997
60998
61011
61012
61015
61019
61020
61022
61030
61031
61032
61033
61036
61037
61038
61042
61044
61049
61051
61062
61065
61067
61070
61077
61078
61089
61093
61106
61107
61108
61109
61111
61121
61133
61162
61163
61164
61165
61167
61168
61170
61171
61172
61173
61176
61180
61187
61193
61195
61197
61202
61203
61204
61206
61210
61219
61221
61223
61228
61229
61230
61235
61240
61247
61256
61257
61258
61261
61264
61281
61286
61295
61296
61303
61304
61318
61326
61327
61331
61335
61341
61344
61346
61349
61351
61353
61358
61375
61376
61380
61381
6138

66081
66083
66085
66088
66093
66094
66104
66105
66115
66116
66117
66119
66121
66123
66124
66134
66140
66144
66149
66150
66151
66154
66155
66157
66164
66166
66171
66172
66175
66179
66181
66185
66188
66189
66190
66198
66199
66201
66203
66215
66216
66220
66221
66223
66231
66232
66233
66235
66236
66237
66240
66245
66246
66247
66250
66252
66257
66259
66260
66264
66265
66269
66275
66277
66278
66284
66288
66289
66294
66300
66301
66302
66303
66313
66321
66322
66327
66328
66337
66340
66342
66344
66346
66348
66355
66357
66358
66362
66365
66367
66369
66371
66378
66384
66393
66401
66406
66410
66412
66413
66422
66425
66437
66440
66441
66445
66446
66452
66453
66454
66457
66462
66465
66467
66473
66474
66476
66478
66483
66484
66486
66492
66503
66509
66510
66512
66519
66525
66533
66541
66542
66549
66552
66564
66569
66570
66584
66586
66590
66593
66594
66597
66598
66599
66604
66605
66607
66609
66615
66622
66623
66625
66631
66638
66640
66647
66653
66655
66656
66667
66670
66674
66675
66677
66680
66683
6668

71241
71243
71246
71250
71254
71260
71261
71262
71263
71266
71267
71268
71269
71276
71279
71280
71283
71290
71291
71294
71295
71297
71300
71301
71302
71308
71309
71317
71318
71321
71322
71323
71327
71330
71335
71337
71338
71341
71346
71348
71349
71360
71365
71366
71367
71370
71373
71377
71378
71385
71387
71392
71393
71396
71399
71402
71404
71416
71423
71424
71427
71435
71438
71441
71445
71455
71461
71467
71469
71476
71479
71481
71486
71491
71492
71499
71500
71501
71504
71511
71514
71518
71525
71528
71532
71534
71537
71538
71540
71542
71543
71544
71547
71550
71551
71552
71554
71555
71556
71558
71566
71570
71573
71576
71584
71589
71590
71597
71598
71600
71603
71604
71605
71607
71615
71616
71619
71620
71621
71623
71628
71632
71634
71650
71656
71666
71668
71669
71671
71674
71691
71697
71702
71704
71705
71706
71707
71708
71710
71713
71716
71717
71722
71726
71728
71730
71732
71733
71738
71742
71746
71749
71752
71769
71770
71773
71775
71777
71781
71799
71800
71803
71806
71808
71812
71814
7182

75906
75910
75911
75912
75914
75920
75924
75925
75931
75937
75938
75943
75944
75952
75953
75955
75958
75960
75962
75965
75977
75981
75983
75986
75990
76000
76002
76010
76011
76020
76024
76030
76031
76033
76045
76054
76057
76061
76062
76063
76066
76069
76075
76084
76091
76093
76094
76096
76107
76110
76111
76112
76114
76115
76116
76117
76122
76123
76125
76126
76127
76135
76137
76145
76146
76147
76150
76152
76154
76155
76156
76159
76161
76165
76167
76172
76173
76176
76177
76187
76189
76190
76192
76196
76200
76201
76207
76211
76212
76214
76215
76217
76219
76221
76229
76235
76238
76241
76242
76251
76258
76259
76260
76264
76266
76267
76268
76270
76271
76276
76277
76279
76280
76289
76293
76294
76302
76305
76307
76317
76324
76328
76336
76342
76345
76348
76358
76360
76368
76371
76373
76374
76377
76378
76380
76381
76390
76393
76396
76397
76407
76410
76414
76422
76423
76426
76427
76429
76432
76434
76438
76439
76440
76442
76448
76457
76460
76465
76475
76486
76487
76488
76490
76492
76493
76494
7649

81553
81554
81556
81558
81560
81562
81566
81567
81568
81577
81581
81582
81583
81586
81587
81588
81593
81594
81600
81620
81621
81623
81630
81633
81635
81639
81641
81642
81658
81666
81668
81669
81671
81672
81676
81677
81682
81687
81690
81695
81700
81710
81713
81714
81716
81727
81734
81741
81746
81750
81757
81759
81761
81763
81768
81777
81778
81788
81794
81799
81801
81802
81814
81815
81816
81821
81826
81828
81834
81841
81843
81849
81852
81854
81860
81873
81882
81883
81884
81892
81893
81894
81906
81912
81913
81915
81918
81925
81928
81932
81933
81942
81948
81953
81961
81966
81967
81977
81982
81988
81990
81998
81999
82004
82021
82024
82030
82031
82032
82037
82042
82044
82050
82055
82058
82059
82060
82062
82064
82084
82086
82089
82093
82095
82098
82102
82105
82109
82110
82119
82122
82125
82126
82127
82130
82133
82137
82139
82142
82144
82149
82152
82157
82164
82168
82170
82175
82177
82183
82184
82187
82190
82200
82202
82204
82208
82216
82223
82225
82228
82232
82235
82237
82239
82240
82241
8224

In [25]:
data.loc[missing, :].head(30)

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Injury Severity,Aircraft Damage,...,Weather Condition,Broad Phase of Flight,Report Status,Publication Date,,latitude_rounded,longitude_rounded,airportcode,airportname,airportname_cleanedletter
0,20190917X60952,Accident,GAA19CA552,09/17/2019,"Palmer, AK",United States,61.355278,-148.409444,Non-Fatal,Substantial,...,,,Preliminary,09/18/2019,,61.4,-148.4,AK53,Maud Road Strip Airport,NA
2,20190917X92718,Incident,ENG19IA034,09/16/2019,"Côte-d'Or, France",France,,,Incident,,...,,,Preliminary,09/18/2019,,999999.0,999999.0,,,
3,20190916X82539,Accident,CEN19LA321,09/15/2019,,United States,28.866945,-89.302222,Non-Fatal,Destroyed,...,,MANEUVERING,Preliminary,09/18/2019,,28.9,-89.3,5LA6,E-8 Tank Battery Seaplane Base,
6,20190912X21831,Accident,CEN19LA316,09/12/2019,"Eustace, TX",United States,32.276667,-96.006667,Fatal(1),Substantial,...,VMC,CRUISE,Preliminary,09/18/2019,,32.3,-96.0,,N/A,NA
9,20190909X51212,Accident,CEN19LA309,09/09/2019,"WALKERTON, IN",United States,41.505000,-86.515556,Non-Fatal,Substantial,...,,MANEUVERING,Preliminary,09/11/2019,,41.5,-86.5,17II,Dreessen Field,
11,20190911X92611,Accident,WPR19FA256,09/08/2019,"La Grande, OR",United States,45.146389,-118.293333,Fatal(1),Destroyed,...,,CRUISE,Preliminary,09/17/2019,,45.1,-118.3,KLGD,La Grande/Union County Airport,NA
13,20190909X13121,Accident,CEN19FA307,09/08/2019,"Lakeview, SD",United States,43.039444,-100.805556,Fatal(2),Substantial,...,VMC,UNKNOWN,Preliminary,09/16/2019,,43.0,-100.8,,N/A,NA
15,20190909X03241,Accident,GAA19CA528,09/07/2019,"Lakeport, CA",United States,39.039444,-122.907778,Unavailable,Substantial,...,,,Preliminary,09/10/2019,,39.0,-122.9,,,
18,20190907X85739,Accident,CEN19FA305,09/07/2019,"Carpenter, WY",United States,41.030000,-104.246389,Fatal(1),Destroyed,...,VMC,CRUISE,Preliminary,09/10/2019,,41.0,-104.2,,N/A,NA
23,20190905X13813,Accident,WPR19FA250,09/04/2019,"Henefer, UT",United States,41.015278,-111.580834,Fatal(2),Substantial,...,VMC,MANEUVERING,Preliminary,09/17/2019,,41.0,-111.6,,N/A,NA


#### Match name and code

In [26]:
"""
Match name & code using the dictionaries

Output columns: 
1. airportcode_new
2. airportname_new
3. code valid - 'trusted' if from code_dict_trusted; 'risky' if from code_dict_risky; otherwise 'notfound'
4. name valid - 'trusted' if from name_dict_trusted; otherwise 'notfound'
"""


code_cleaned = []
name_cleaned = []
code_bool = []

for i in data.index: 
    code = data.loc[i, 'airportcode']
    name = data.loc[i, 'airportname_cleanedletter']
    
    code_ind = code_dict_trusted.get(code)
    name_ind = name_dict_trusted.get(code)
    if not pd.isna(code_ind):
        code_cleaned.append(airport_code.loc[code_ind, 'ident'])
        name_cleaned.append(airport_code.loc[code_ind, 'name_x'])
        code_bool.append('trusted')
    else: 
        code_ind_r = code_dict_risky.get(code)
        if not pd.isna(code_ind_r) and not pd.isna(name_ind):
            updated = False
            for ind in name_ind: 
                if ind == code_ind_r:      
                    code_cleaned.append(airport_code.loc[code_ind_r, 'ident'])
                    name_cleaned.append(airport_code.loc[code_ind_r, 'name_x'])
                    code_bool.append('trusted')
                    updated = True
                    break
            if not updated: 
                code_cleaned.append(airport_code.loc[code_ind_r, 'ident'])
                name_cleaned.append(airport_code.loc[code_ind_r, 'name_x'])
                code_bool.append('risky')
        elif not pd.isna(code_ind_r): 
            code_cleaned.append(airport_code.loc[code_ind_r, 'ident'])
            name_cleaned.append(airport_code.loc[code_ind_r, 'name_x'])
            code_bool.append('risky')
        elif not pd.isna(name_ind): 
            code_cleaned.append(airport_code.loc[name_ind, 'ident'])
            name_cleaned.append(airport_code.loc[name_ind, 'name_x'])
            code_bool.append('trusted')
        else: 
            code_cleaned.append(code)
            name_cleaned.append(name)
            code_bool.append('notfound')

In [27]:
data['airportcode_new'] = code_cleaned
data['airportname_new'] = name_cleaned
data['code valid'] = code_bool

In [28]:
data.head(20)

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Injury Severity,Aircraft Damage,...,Publication Date,,latitude_rounded,longitude_rounded,airportcode,airportname,airportname_cleanedletter,airportcode_new,airportname_new,code valid
0,20190917X60952,Accident,GAA19CA552,09/17/2019,"Palmer, AK",United States,61.355278,-148.409444,Non-Fatal,Substantial,...,09/18/2019,,61.4,-148.4,AK53,Maud Road Strip Airport,NA,AK53,NA,notfound
1,20190916X74851,Accident,GAA19CA550,09/16/2019,"Reno, NV",United States,39.668056,-119.876389,Unavailable,Substantial,...,09/18/2019,,39.7,-119.9,RTS,Reno/Stead,RENOSTEAD,RTS,RENOSTEAD,notfound
2,20190917X92718,Incident,ENG19IA034,09/16/2019,"Côte-d'Or, France",France,,,Incident,,...,09/18/2019,,999999.0,999999.0,,,,,,notfound
3,20190916X82539,Accident,CEN19LA321,09/15/2019,,United States,28.866945,-89.302222,Non-Fatal,Destroyed,...,09/18/2019,,28.9,-89.3,5LA6,E-8 Tank Battery Seaplane Base,,5LA6,,notfound
4,20190912X05539,Accident,GAA19CA544,09/12/2019,"Hemet, CA",United States,33.734166,-117.022223,Non-Fatal,Substantial,...,09/18/2019,,33.7,-117.0,HMT,Hemet-Ryan,HEMETRYAN,KHMT,Hemet Ryan Airport,trusted
5,20190912X11044,Accident,GAA19CA542,09/12/2019,"Phoenix, AZ",United States,33.688333,-112.082500,Non-Fatal,Substantial,...,09/17/2019,,33.7,-112.1,DVT,Phoenix Deer Valley,PHOENIXDEERVALLEY,EDVT,Ithwiesen Airfield,trusted
6,20190912X21831,Accident,CEN19LA316,09/12/2019,"Eustace, TX",United States,32.276667,-96.006667,Fatal(1),Substantial,...,09/18/2019,,32.3,-96.0,,N/A,NA,,NA,notfound
7,20190911X11037,Accident,CEN19LA317,09/11/2019,"Boca Raton, FL",United States,26.378611,-80.108333,Non-Fatal,Destroyed,...,09/13/2019,,26.4,-80.1,BCT,Boca Raton Airport,BOCARATONAIRPORT,KBCT,Boca Raton Airport,trusted
8,20190912X11052,Accident,CEN19LA315,09/11/2019,"Kingman, KS",United States,37.669167,-98.123889,Non-Fatal,Substantial,...,09/17/2019,,37.7,-98.1,9K8,Kingman Airport,KINGMANAIRPORT,K9K8,Kingman Airport Clyde Cessna Field,trusted
9,20190909X51212,Accident,CEN19LA309,09/09/2019,"WALKERTON, IN",United States,41.505000,-86.515556,Non-Fatal,Substantial,...,09/11/2019,,41.5,-86.5,17II,Dreessen Field,,17II,,notfound


In [29]:
# to give a summary

# bool1 = pd.isna(data['airportcode']) 
# bool2 = (data['airportcode'] == '')
# bool3 = (data['airportcode'] == 'N/A')
# bool4 = pd.isna(data['airportname'])
# bool5 = (data['airportname'] == '')
# bool6 = (data['airportname'] == 'N/A')


both_na = []
for i in data.index: 
    both_na.append(isinvalid(data.loc[i, 'airportcode']) and isinvalid(data.loc[i, 'airportname']))

In [30]:
print('total number of entries = ' + str(len(data)))
# print('number of entries with both null code & name = ' + str(sum(both_na)))
print('with trusted code & name = ' + str(len(data[data['code valid']=='trusted'])))
print('with risky code & name = ' + str(len(data[data['code valid']=='risky'])))
print('total notfound = ' + str(len(data[data['code valid']=='notfound'])))

total number of entries = 83807
with trusted code & name = 47329
with risky code & name = 717
total notfound = 35761


#### Number of accidents per airportcode/airportname per month

In [31]:
data[' Investigation Type '].unique()

array([' Accident ', ' Incident ', '  '], dtype=object)

In [32]:
data = pd.get_dummies(data, columns=[' Investigation Type '], drop_first=True)

In [33]:
event_date = np.array([data.loc[i, ' Event Date '].strip().split('/') for i in  np.arange(len(data))]).T

month = event_date[0]
day = event_date[1]
year = event_date[2]

In [34]:
data['year'] = year
data['month'] = month

In [35]:
data_filtered = data[data['code valid'] != 'notfound']

In [36]:
data_filtered

,Event Id,Accident Number,Event Date,Location,Country,Latitude,Longitude,Injury Severity,Aircraft Damage,Aircraft Category,...,airportcode,airportname,airportname_cleanedletter,airportcode_new,airportname_new,code valid,Investigation Type _ Accident,Investigation Type _ Incident,year,month
4,20190912X05539,GAA19CA544,09/12/2019,"Hemet, CA",United States,33.734166,-117.022223,Non-Fatal,Substantial,Airplane,...,HMT,Hemet-Ryan,HEMETRYAN,KHMT,Hemet Ryan Airport,trusted,1,0,2019,09
5,20190912X11044,GAA19CA542,09/12/2019,"Phoenix, AZ",United States,33.688333,-112.082500,Non-Fatal,Substantial,Airplane,...,DVT,Phoenix Deer Valley,PHOENIXDEERVALLEY,EDVT,Ithwiesen Airfield,trusted,1,0,2019,09
7,20190911X11037,CEN19LA317,09/11/2019,"Boca Raton, FL",United States,26.378611,-80.108333,Non-Fatal,Destroyed,Airplane,...,BCT,Boca Raton Airport,BOCARATONAIRPORT,KBCT,Boca Raton Airport,trusted,1,0,2019,09
8,20190912X11052,CEN19LA315,09/11/2019,"Kingman, KS",United States,37.669167,-98.123889,Non-Fatal,Substantial,Airplane,...,9K8,Kingman Airport,KINGMANAIRPORT,K9K8,Kingman Airport Clyde Cessna Field,trusted,1,0,2019,09
10,20190909X25123,CEN19LA308,09/09/2019,"WHITEHALL, MI",United States,43.405556,-86.335555,Fatal(1),Destroyed,Airplane,...,KFFX,Fremont Municipal Airport,FREMONTMUNICIPALAIRPORT,KFFX,Fremont Municipal Airport,trusted,1,0,2019,09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83791,20020917X01894,CHI82FEC08,01/02/1982,"YPSILANTI, MI",United States,,,Non-Fatal,Substantial,Airplane,...,YIP,WILLOW RUN,WILLOWRUN,AYIP,Iropena Airstrip,trusted,1,0,1982,01
83792,20020917X01776,CHI82DA020,01/02/1982,"CHARLOTTE, MI",United States,,,Non-Fatal,Substantial,Airplane,...,49G,TINKERBELL,TINKERBELL,49G,Wend Valley Airport,trusted,1,0,1982,01
83793,20020917X01657,ATL82DA027,01/02/1982,"CHAMBLEE, GA",United States,,,Non-Fatal,Substantial,Helicopter,...,PDK,,,KPDK,DeKalb Peachtree Airport,trusted,1,0,1982,01
83796,20020909X01561,NYC82DA015,01/01/1982,"EAST HANOVER, NJ",United States,,,Non-Fatal,Substantial,Airplane,...,N58,HANOVER,HANOVER,IN58,Sealscott Airport,trusted,1,0,1982,01


In [37]:
data_filtered.columns

Index(['Event Id ', ' Accident Number ', ' Event Date ', ' Location ',
       ' Country ', ' Latitude ', ' Longitude ', ' Injury Severity ',
       ' Aircraft Damage ', ' Aircraft Category ', ' Registration Number ',
       ' Make ', ' Model ', ' Amateur Built ', ' Number of Engines ',
       ' Engine Type ', ' FAR Description ', ' Schedule ',
       ' Purpose of Flight ', ' Air Carrier ', ' Total Fatal Injuries ',
       ' Total Serious Injuries ', ' Total Minor Injuries ',
       ' Total Uninjured ', ' Weather Condition ', ' Broad Phase of Flight ',
       ' Report Status ', ' Publication Date ', ' ', 'latitude_rounded',
       'longitude_rounded', 'airportcode', 'airportname',
       'airportname_cleanedletter', 'airportcode_new', 'airportname_new',
       'code valid', ' Investigation Type _ Accident ',
       ' Investigation Type _ Incident ', 'year', 'month'],
      dtype='object')

In [38]:
data_filtered['airportcode_new'] = data_filtered['airportcode_new'].apply(str)
data_filtered['airportname_new'] = data_filtered['airportname_new'].apply(str)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
output_df = data_filtered[['airportcode_new', 'airportname_new', 'year', 'month', ' Investigation Type _ Accident ', 
      ' Investigation Type _ Incident ', 'Event Id ']].groupby(by=['airportcode_new', 'airportname_new', 'year', 'month']).sum()

In [40]:
output_df.to_csv("NTSB_AIDS_full_output_new.csv")